In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import onnxruntime

img_path = 'ade20k.jpg'

img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [4]:
model = ResNet50(weights='imagenet')

preds = model.predict(x)
print('Keras Predicted:', decode_predictions(preds, top=3)[0])
print(os.path.join("/tmp", model.name))
model.save(f'{model.name}.h5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


Keras Predicted: [('n04285008', 'sports_car', np.float32(0.34477845)), ('n02974003', 'car_wheel', np.float32(0.28764325)), ('n03100240', 'convertible', np.float32(0.10070887))]
/tmp\resnet50


In [ ]:

import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = model.name + ".onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x000001A09201DEA0>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


['predictions']


In [12]:

providers = ['CPUExecutionProvider']
session = rt.InferenceSession(output_path, providers=providers)
onnx_pred = session.run(output_names, {"input": x})

print('ONNX Predicted:', decode_predictions(onnx_pred[0], top=3)[0])

ONNX Predicted: [('n04285008', 'sports_car', np.float32(0.3447771)), ('n02974003', 'car_wheel', np.float32(0.2876441)), ('n03100240', 'convertible', np.float32(0.10070868))]
